In [16]:
import duckdb
import pandas as pd
from IPython.display import display
import plotly.express as px
from eda_support_functions import *


In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [18]:
wikipedia_movie_financial_data = pd.read_csv('../data/retrieved_data/wikipedia/movie_financial_data.csv').drop(columns='Unnamed: 0')
wikipedia_movie_financial_data_usd = pd.read_csv('../data/retrieved_data/wikipedia/movie_financial_data_usd.csv')
# make sure we are 


movie = pd.read_csv('../data/retrieved_data/tmdb/movies.csv')
mojo = pd.read_csv('../data/manual_data/parsed_mojo.csv')
mojo = mojov

In [19]:
manual_adjustments = pd.read_csv('../data/manual_data/manual_adjustments.csv')


In [20]:
manual_adjustments = manual_adjustments[["movie_id", "revenue_fix","budget_fix"]].fillna(0)

In [21]:
manual_adjustments

,movie_id,revenue_fix,budget_fix
0,280106,1607846.0,0.0
1,21435,84200000.0,35000000.0
2,13748,5600000.0,0.0
3,37905,688225.0,0.0
4,10407,0.0,26000000.0
...,...,...,...
9413,341742,21100000.0,10000000.0
9414,14424,0.0,5000000.0
9415,32834,0.0,4500000.0
9416,280422,0.0,0.0


In [22]:
# Print how many movies in wikipedia_movie_financial_data
print(f'Number of movies in wikipedia_movie_financial_data: {len(wikipedia_movie_financial_data)}')
unique_imdb_ids = wikipedia_movie_financial_data['tmdb_id'].unique()
print(f'Number of unique imdb_ids in wikipedia_movie_financial_data: {len(unique_imdb_ids)}')


Number of movies in wikipedia_movie_financial_data: 43355
Number of unique imdb_ids in wikipedia_movie_financial_data: 43355


In [23]:
# #load json from url into dataframe
# import requests
# import json
# url = 'https://ffbe7889f116f85ae79fd90903cf7cccfd41509ca2fe525c2cbda26-apidata.googleusercontent.com/download/storage/v1/b/movie-dataset-291206-movie-data-results/o/20240714_094035_aggregated_results.json?jk=AYBlUPBJApueiHReA-hfruhHxgh8ghoMyS56g-SXJFQSLSg4fymkM4dgnxV6XZyyLQuP54YFFyIEjVmx_Q4oG5jGu3cxPMi2UMXB2FR_DARj-6blknNhuXaBHp6tCXf0HYtxwrC7yklq7U2a8NBQFFf9sLik7i5I1JNWLEe7tDOrU1rAKTNmstj9T9EAWAkX8ULAKB51HFE_mf1TN6NsCEnt_wvWpnY7kufRoI2c-NiLrckk5RGoUnXIiwQF5PP1BIRyIMTHrPm_qeE9s8hnSViNFc0rfhHO0BsIe559FqnDEdIhux07QUfh2DARX8dXKo0Mmb_YHCx368SaexGNydOzAGkfPwhLV9T914KDMwOOOp-H2BPm1kKFKyUc9zdzVQx-dA4RSzvvZn7Ti6n5xPpDhHfgMQe-zI0aRezkwpS6UvdCilkarNR5Mz-5M9Pst9Mqil25h-_2D88ALyROA5090E-iyursRs8smId5NE21LkTHcIBZb1-L9XMhsaJw701dzR7AaKZO2vdh0IouEiBJyWCFTmjDiZ2QJXlM6ssEWyMEQZGl5f17BiXTI2RzO-XnuaWPgJ3uulaOa7OijKsMQYt9mZg6G5usx7B8oDwV2TtkTepqBJvMztVC1iVJ2HgE3VR-tPNkP3TXvsELbYcX4OM1XlOJ-ombTOamS-nQsbH7NKLUCxF3yhOGtFbtMaHt7A9Uv6jr0XKkXiw3YrW9H-AdVMrzqbHKzU5PrWJV0NC_JVkI6fK8t_XweMN_Du--nZOTtdsYLkeiR3iZ2LeBxneKkxgbnAwKBnv0gVRtcrP54C55EwKWkdrf7yAnl6DQBBRZRCZCcZwliv01gHXG4AcKbYLCl3dPotdBR_VvGMEkq6MZ1GQI639GSYhl_38fUBix884nosRXiog4V5m8c__hz1boaESxkpo89QO8Y-IFgFYm1_uMfzxxwHAn0hkRjD692jfEowJcn_2p7i3lN-6CnM6aZONYoYCfTVMIfFm8IuZKWyCHYABunDXgDd5iquPYOrLFFFOFPcqd9RF-jQJQ_P49NoA_ITCEGQK44veox7bbpI0Pbju6zHpTswIFEESbyLbHFmFjLiVMQGVCP51J5Do79r-aT7DkKNTskdrm4mzSpLN9aPEN5CxgqB_cNj9xaxB8W36AIximjr08S9_6un6P4dxUFFmconT0bWxel1kcQZV-zyvt6mSGuhegE-fXPFY8bVaI4dYtQYURdhf2RVC8&isca=1'
# response = requests.get(url)
# data = response.json()
# # print(data)
# mojo = pd.read_json(json.dumps(data), orient='index')

# mojo.to_csv('../data/manual_data/parsed_mojo.csv')

# Wikipedia

In [24]:
# Check wikipedia_movie_financial_data for duplicates on tmdb_id
display(wikipedia_movie_financial_data[wikipedia_movie_financial_data.duplicated(subset=['tmdb_id'], keep=False)])

# How many unique tmdb_ids are duplicated?
print(wikipedia_movie_financial_data[wikipedia_movie_financial_data.duplicated(subset=['tmdb_id'], keep=False)]['tmdb_id'].nunique())



,url,tmdb_id,budget_original_string,budget_currency,budget_min,budget_max,budget_avg,budget_error,box_office_original_string,box_office_currency,box office_min,box_office_max,box_office_avg,box_office_error,external_links,error,box_office_min


0


In [25]:
wikipedia_movie_financial_data.groupby(by=['budget_currency']).size().sort_values(ascending=False).head(20)

budget_currency
USD    10518
INR      828
EUR      601
GBP      447
AUD      217
CAD      110
BRL      106
JPY       79
SEK       54
KRW       44
RUB       43
HKD       39
CZK       31
DEM       28
ESP       27
PHP       24
NOK       19
NGN       16
DKK       14
CHF       14
dtype: int64

In [26]:
wikipedia_movie_financial_data.groupby(by=['box_office_currency']).size().sort_values(ascending=False).head(20)

box_office_currency
USD    13193
INR      919
HKD      290
JPY      254
AUD      176
GBP      167
EUR      123
ITL      115
BRL       86
PHP       57
SEK       32
RUB       31
ESP       28
NGN       24
CHF       23
CZK       18
CAD       17
DEM       10
TRY       10
KRW        9
dtype: int64

In [27]:
wikipedia_movie_financial_data[(wikipedia_movie_financial_data['budget_currency'].isnull()) & (wikipedia_movie_financial_data['budget_original_string'].notnull())]['budget_original_string'].value_counts()[:70]

budget_original_string
Unknown                                                               8
IL3 million                                                           7
P208,120,000                                                          2
MYR 2.5 million                                                       2
ƒ 5 million                                                           1
MYR 3 million                                                         1
-                                                                     1
Rs 1,50,000                                                           1
Undisclosed                                                           1
Kadr Film                                                             1
7.6 million [1]                                                       1
3 million[1]                                                          1
12 million FF                                                         1
Rp. 80 million                           

In [28]:
print(f"Number of revenue_mojo columns not with 0 values: {(mojo['revenue_mojo'] != 0).sum()}")
print(f"Number of cases with error: {(~mojo['error'].isna()).sum()}")



Number of revenue_mojo columns not with 0 values: 5931
Number of cases with error: 23


In [29]:
mojo = mojo[['imdb_id', 'revenue_mojo', 'error']]
movie = movie.merge(mojo, on='imdb_id', how='left')
movie['revenue_mojo'] = movie['revenue_mojo'].fillna(0)

wikipedia_movie_financial_data_usd_raw = wikipedia_movie_financial_data_usd.copy()
wikipedia_movie_financial_data_usd = wikipedia_movie_financial_data_usd[['movie_id', 'budget_avg_usd', 'box_office_avg_usd', 'url']].rename(columns={'budget_avg_usd': 'budget_wiki', 'box_office_avg_usd': 'revenue_wiki','url' : 'url_wiki'})
movie = movie.merge(wikipedia_movie_financial_data_usd, on='movie_id', how='left')
movie = movie.merge(manual_adjustments, on='movie_id', how='left')
movie['budget_wiki'] = movie['budget_wiki'].fillna(0).astype(int)
movie['revenue_wiki'] = movie['revenue_wiki'].fillna(0).astype(int)
movie['revenue_fix'] = movie['revenue_fix'].fillna(0).astype(int)
movie['budget_fix'] = movie['budget_fix'].fillna(0).astype(int)




In [30]:
# Replace the 0 values with the values from the wikipedia dataset
movie['budget_synthetic'] = movie['budget_fix'].replace(0, np.nan)
movie['budget_synthetic'] = movie['budget_synthetic'].fillna(movie['budget']).replace(0, np.nan)
movie['budget_synthetic'] = movie['budget_synthetic'].fillna(movie['budget_wiki']).fillna(0).astype(int)

# Create synthetic box office
movie['revenue_synthetic'] = movie['revenue_fix'].replace(0, np.nan)
movie['revenue_synthetic'] = movie['revenue_synthetic'].fillna(movie['revenue_mojo']).replace(0, np.nan)
movie['revenue_synthetic'] = movie['revenue_synthetic'].fillna(movie['revenue']).replace(0, np.nan)
movie['revenue_synthetic'] = movie['revenue_synthetic'].fillna(movie['revenue_wiki']).fillna(0).astype(int)

In [31]:
print("TMDB Data")
print(f"Number of tmdb revenue column not 0: {(movie['revenue'] != 0).sum()}")
print(f"Number of tmdb budget column not 0: {(movie['budget'] != 0).sum()}")
print(f"Number of revenue and budget not 0 : {((movie['revenue'] != 0) & (movie['budget'] != 0)).sum()}")

print("Mojo Enriched")
print(f"Number of mojo revenue column not 0: {(movie['revenue_mojo'] != 0).sum()}")
print(f"Number of revenue not 0: {((movie['revenue'] != 0) | (movie['revenue_mojo'] != 0)).sum()}")
print(f"Number of revenue and budget not 0 : {(((movie['revenue'] != 0) | (movie['revenue_mojo'] != 0)) & (movie['budget'] != 0)).sum()}")

print("Wikipedia Enriched")
print(f"Number of revenue column not 0: {(movie['revenue_synthetic'] != 0).sum()}")
print(f"Number of budget column not 0: {(movie['budget_synthetic'] != 0).sum()}")
print(f"Number of revenue and budget not 0 : {((movie['revenue_synthetic'] != 0) & (movie['budget_synthetic'] != 0)).sum()}")


TMDB Data
Number of tmdb revenue column not 0: 11499
Number of tmdb budget column not 0: 11700
Number of revenue and budget not 0 : 7634
Mojo Enriched
Number of mojo revenue column not 0: 5908
Number of revenue not 0: 14594
Number of revenue and budget not 0 : 7987
Wikipedia Enriched
Number of revenue column not 0: 19784
Number of budget column not 0: 16558
Number of revenue and budget not 0 : 12183


In [32]:
compare = movie[(movie['revenue'] != 0) & (movie['revenue_mojo'] != 0)].copy()
compare['revenue_diff'] = compare['revenue'] - compare['revenue_mojo']

# Calulate metrics
compare['revenue_diff'] = compare['revenue'] - compare['revenue_mojo']
compare['revenue_ratio'] = compare['revenue'] / compare['revenue_mojo']
compare['revenue_diff_ratio'] = compare['revenue_diff'] / compare['revenue_mojo']

# Display
display(compare[['imdb_id', 'revenue', 'revenue_mojo', 'revenue_diff', 'revenue_ratio', 'revenue_diff_ratio']].sort_values(by='revenue_diff_ratio', ascending=False).head(10))

# Describe
compare[['revenue_diff_ratio','revenue_ratio', 'revenue_diff']].describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.97, 0.98, 0.99]).style.format("{:.2f}")

,imdb_id,revenue,revenue_mojo,revenue_diff,revenue_ratio,revenue_diff_ratio
61908,tt0080516,12000000,147.0,11999853.0,81632.653061,81631.653061
46271,tt0066769,12376563,430.0,12376133.0,28782.704651,28781.704651
55997,tt0070723,3600000,210.0,3599790.0,17142.857143,17141.857143
62881,tt0080310,8200000,948.0,8199052.0,8649.789030,8648.789030
48623,tt0074695,1509000,201.0,1508799.0,7507.462687,7506.462687
27475,tt0068638,36734619,9588.0,36725031.0,3831.311952,3830.311952
5116,tt0082464,1400000,394.0,1399606.0,3553.299492,3552.299492
23584,tt5125414,7082000,3403.0,7078597.0,2081.104907,2080.104907
4298,tt0080117,23000000,14492.0,22985508.0,1587.082528,1586.082528
8320,tt0071222,4000000,2686.0,3997314.0,1489.203276,1488.203276


,revenue_diff_ratio,revenue_ratio,revenue_diff
count,2813.00,2813.00,2813.00
mean,57.87,58.87,82323.88
std,1681.24,1681.24,9841623.61
min,-1.00,0.00,-231142888.00
1%,-1.00,0.00,-21120563.00
5%,-0.65,0.35,-3324037.80
10%,-0.26,0.74,-1073131.80
25%,-0.01,0.99,-37633.00
50%,0.00,1.00,0.00
75%,0.00,1.00,0.00


In [33]:
null_percentage_all = movie.isnull().mean() * 100

zero_percentage_all = (movie == 0).mean() * 100

null_zero_percentage_all = pd.DataFrame({'Null Percentage': null_percentage_all, 'Zero Percentage': zero_percentage_all})
print(null_zero_percentage_all)

                   Null Percentage  Zero Percentage
movie_id                  0.000000         0.000000
title                     0.000000         0.000000
original_title            0.000000         0.000000
original_language         0.000000         0.000000
overview                  5.789014         0.000000
tagline                  64.095087         0.000000
release_date              0.241271         0.000000
runtime                   0.000000         5.578827
budget                    0.000000        82.681804
revenue                   0.000000        82.979322
poster_path               3.608698         0.000000
imdb_id                   0.458858         0.000000
facebook_id              95.760742         0.000000
instagram_id             98.291863         0.000000
twitter_id               97.945499         0.000000
wikidata_id              88.629198         0.000000
revenue_mojo              0.000000        91.255051
error                    99.965956         0.000000
budget_wiki 

In [34]:
movie = movie[(movie['revenue_synthetic'] != 0) & (movie['budget_synthetic'] != 0)]

In [35]:
movie[['budget', 'budget_wiki', 'budget_synthetic', 'revenue', 'revenue_mojo', 'revenue_wiki','revenue_synthetic']].describe().drop('count').style.format({
    'budget': "${:,.0f}",
    'budget_wiki': "${:,.0f}",
    'budget_synthetic': "${:,.0f}",
    'revenue': "${:,.0f}",
    'revenue_mojo': "${:,.0f}",
    'revenue_wiki': "${:,.0f}",
    'revenue_synthetic': "${:,.0f}"
})

,budget,budget_wiki,budget_synthetic,revenue,revenue_mojo,revenue_wiki,revenue_synthetic
mean,"$15,765,264","$16,492,973","$17,900,084","$41,711,358","$11,753,012","$45,109,829","$43,922,431"
std,"$29,279,919","$30,882,198","$29,500,371","$118,984,042","$64,961,723","$122,449,255","$119,119,185"
min,$0,$0,$1,$0,$0,$0,$2
25%,$0,"$1,721,240","$3,000,000",$0,$0,"$1,022,820","$1,354,824"
50%,"$5,000,000","$7,000,000","$8,000,000","$3,764,219",$0,"$7,800,000","$7,800,000"
75%,"$19,000,000","$19,775,000","$20,000,000","$30,673,624","$7,755","$35,800,000","$33,298,228"
max,"$379,000,000","$1,107,215,353","$500,000,000","$2,923,706,026","$2,923,706,026","$2,923,000,000","$2,923,706,026"


In [44]:
movie['synthetic_ratio'] = movie['revenue_synthetic'] / movie['budget_synthetic']

In [45]:
movie

,movie_id,title,original_title,original_language,overview,tagline,release_date,runtime,budget,revenue,poster_path,imdb_id,facebook_id,instagram_id,twitter_id,wikidata_id,revenue_mojo,error,budget_wiki,revenue_wiki,url_wiki,revenue_fix,budget_fix,budget_synthetic,revenue_synthetic,synthetic_ratio
6,55895,Monogamy,Monogamy,en,"Exhibitionism, voyeurism, jealousy, lust. Broo...",NaN,2010-04-24,94,1000000,0,/7NmXl4GbSJW1kbcZ1OPKt0UC6E7.jpg,tt1502714,NaN,NaN,NaN,NaN,21527.0,NaN,1000000,0,https://en.wikipedia.org/wiki/Monogamy_(film),0,0,1000000,21527,0.021527
25,14854,Lone Wolf McQuade,Lone Wolf McQuade,en,The archetypical renegade Texas Ranger wages w...,The 'Mad Dog' Criminal... The 'Lone Wolf' Lawm...,1983-04-15,107,5000000,12232628,/dTUTDPilI2Ozi5GeoBRczidQTaZ.jpg,tt0085862,NaN,NaN,NaN,NaN,12232628.0,NaN,5000000,13616314,https://en.wikipedia.org/wiki/Lone_Wolf_McQuade,0,0,5000000,12232628,2.446526
35,296690,Kill Dil,किल दिल,hi,"Once upon a time in North India, two killers –...",NaN,2014-11-13,127,0,0,/rs93Rh0oeJ4eYCpfn0GvkrtpSbQ.jpg,tt3380264,NaN,NaN,NaN,NaN,27256.0,NaN,6431385,8376879,https://en.wikipedia.org/wiki/Kill_Dil,0,0,6431385,27256,0.004238
39,55903,Betty Fisher and Other Stories,Betty Fisher et autres histoires,fr,When bestselling novelist Betty Fisher loses h...,Crimes of passion are not always between lovers.,2001-09-01,103,0,0,/hYMsIPY8APyppmc1wHztD3zLR0g.jpg,tt0269329,NaN,NaN,NaN,NaN,676239.0,NaN,9000000,468,https://en.wikipedia.org/wiki/Alias_Betty,0,0,9000000,676239,0.075138
43,34544,The A-Team,The A-Team,en,"A man who loves when a plan comes together, Co...",There is no plan B.,2010-06-09,117,110000000,177238796,/k3mKy2zYfzqGg9o5JeDyzSEPPHO.jpg,tt0429493,officialateammovie,NaN,NaN,Q471746,177238796.0,NaN,55000050,177200000,https://en.wikipedia.org/wiki/The_A-Team_(film),0,0,110000000,177238796,1.611262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67530,55890,Bereavement,Bereavement,en,"In 1989, six year old Martin Bristoll was kidn...",There are some evils...so unspeakable...they w...,2010-07-16,103,2000000,10000000,/5Zd0NzvT3x8EhuE5lYMJz5ShcH1.jpg,tt1100051,NaN,NaN,NaN,NaN,43712.0,NaN,2000000,10000000,https://en.wikipedia.org/wiki/Bereavement_(film),0,0,2000000,43712,0.021856
67532,34491,The Wedding Game,大囍事,zh,A black comedy centered on a recently engaged ...,NaN,2009-01-25,103,1000000,0,/igeZ2mn1IjlbhKPbaxHU5cNw4rX.jpg,tt1314659,NaN,NaN,NaN,NaN,1949614.0,NaN,1500000,0,https://en.wikipedia.org/wiki/The_Wedding_Game,0,0,1000000,1949614,1.949614
67533,14839,Blue Collar,Blue Collar,en,Fed up with mistreatment at the hands of both ...,"""Blue Collar"" is the American Dream. If you're...",1978-02-10,114,1700000,6521083,/7WBBcgzESyX1qEq9x7tg22dlORL.jpg,tt0077248,NaN,NaN,NaN,Q885666,6521083.0,NaN,1700000,6500000,https://en.wikipedia.org/wiki/Blue_Collar_(film),0,0,1700000,6521083,3.835931
67546,14846,Little Black Book,Little Black Book,en,Determined to learn about her boyfriend's past...,Have you ever been tempted to look inside his...,2004-08-06,111,35000000,22035509,/votbcPlp3UO7JS7gRa86VDYLWL1.jpg,tt0361841,NaN,NaN,NaN,NaN,22035509.0,NaN,35000000,22000000,https://en.wikipedia.org/wiki/Little_Black_Boo...,0,0,35000000,22035509,0.629586


In [46]:
movie[['movie_id','title','url_wiki','original_language','revenue', 'revenue_wiki', 'revenue_mojo', 'revenue_synthetic','budget', 'budget_wiki', 'budget_synthetic','synthetic_ratio']].sort_values(by='synthetic_ratio', ascending=False).to_clipboard()

In [47]:
movie[['budget', 'budget_wiki','budget_synthetic']].corr().style.format("{:.2f}").background_gradient(cmap='coolwarm')

,budget,budget_wiki,budget_synthetic
budget,1.00,0.78,0.95
budget_wiki,0.78,1.00,0.83
budget_synthetic,0.95,0.83,1.00


In [48]:
movie[['revenue', 'revenue_mojo', 'revenue_wiki','revenue_synthetic']].corr().style.format("{:.2f}").background_gradient(cmap='coolwarm')

,revenue,revenue_mojo,revenue_wiki,revenue_synthetic
revenue,1.00,0.49,0.96,0.99
revenue_mojo,0.49,1.00,0.48,0.50
revenue_wiki,0.96,0.48,1.00,0.96
revenue_synthetic,0.99,0.50,0.96,1.00


In [49]:
movie[['runtime', 'budget', 'revenue']].describe().drop('count').style.format({
    'runtime': "{:.0f}",
    'budget': "${:,.0f}",
    'revenue': "${:,.0f}"
})

,runtime,budget,revenue
mean,109,"$15,765,264","$41,711,358"
std,21,"$29,279,919","$118,984,042"
min,0,$0,$0
25%,94,$0,$0
50%,104,"$5,000,000","$3,764,219"
75%,118,"$19,000,000","$30,673,624"
max,360,"$379,000,000","$2,923,706,026"


In [50]:
mapped_companies =  duckdb.sql('SELECT * FROM production_mapping LEFT JOIN production using(company_id)').to_df()
plot_and_export_categorical_distribution(mapped_companies, 'company_name', 100)

CatalogException: Catalog Error: Table with name production_mapping does not exist!
Did you mean "pg_am"?

In [51]:
keyword_stats = duckdb.sql('SELECT * FROM keyword_mapping LEFT JOIN keyword_d USING(keyword_id)').to_df()
plot_and_export_categorical_distribution(keyword_stats, 'keyword_name', 150)

CatalogException: Catalog Error: Table with name keyword_mapping does not exist!
Did you mean "pg_am"?

In [52]:
duckdb.sql('SELECT COUNT(DISTINCT genre_id) FROM genre_mapping LIMIT 5')

CatalogException: Catalog Error: Table with name genre_mapping does not exist!
Did you mean "pg_am"?

In [53]:
genre_stats = duckdb.sql('SELECT * FROM genre_mapping LEFT JOIN genre USING(genre_id)').to_df()
plot_and_export_categorical_distribution(genre_stats, 'name', 100)

CatalogException: Catalog Error: Table with name genre_mapping does not exist!
Did you mean "pg_am"?

In [54]:
plot_and_export_categorical_distribution(movie, 'original_language', 60)

,grouped,total
0,en,8457
1,fr,1115
2,hi,633
3,Other,518
4,ru,229
5,es,221
6,ko,189
7,zh,168
8,it,118
9,pt,110


In [55]:
plot_and_export_categorical_distribution(movie, 'ageCert', 60)

KeyError: 'ageCert'